In [1]:
# March 27 2024
# explore the addresses from address base further 
# here I'm trying to use additional columns (SAO and PAO) to create a full address string
    # Bin had said that there might be multiple combinations and also we are not getting the full address
    # if we ignore these parts 
# work in progress: I am processing the data by adding an additional column to the end

# May 10 add column for postcode sector

In [2]:
import pandas as pd
import time
import numpy as np
import copy
import re

In [3]:
#create a function to iterate through csv. use for OSAddressBase file
def read_csv(file_name, columns):
    for chunk in pd.read_csv(file_name, chunksize=10000, usecols=columns, 
                            dtype={'urpn':int, 'parent_urpn': int, 'class': str, 'latitude': float, 
                                    'longitude': float, 'country': str, 'legal_name': str, 
                                      'sub_building_name': str, 'building_name': str, 'building_number':'float64', 
                                  'sao_start_number': 'float64', 'sao_start_suffix': str,'sao_end_number': 'float64', 
                                  'sao_end_suffix': str, 'sao_text': str, 'pao_start_number': 'float64', 'pao_start_suffix':str, 
                                  'pao_end_number':'float64', 'pao_end_suffix': str , 'pao_text': str, 'street_description': str} 
                               , 
                             low_memory=False):
    
        #if chunk['country'] == 'E':
        yield chunk


In [4]:
"""
fewer_columns  = ['uprn', 'parent_uprn', 'class', 'latitude', 'longitude', 'country' ]
all_columns = ['uprn', 'parent_uprn', 'class', 'latitude', 'longitude', 'country', 'legal_name','sub_building_name',
           'building_name','building_number','sao_start_number','sao_start_suffix','sao_end_number',
           'sao_end_suffix','sao_text','alt_language_sao_text','pao_start_number','pao_start_suffix',
           'pao_end_number','pao_end_suffix','pao_text','alt_language_pao_text','usrn','usrn_match_indicator',
           'area_name','level','official_flag','os_address_toid','os_address_toid_version','os_roadlink_toid',
           'os_roadlink_toid_version','os_topo_toid','os_topo_toid_version','voa_ct_record','voa_ndr_record',
           'street_description','alt_language_street_description','dependent_thoroughfare','thoroughfare',
           'double_dependent_locality','dependent_locality','locality', 'town_name', 
          'administrative_area','post_town','postcode','postcode_locator' ]

   
                                   'dependent_locality': str, 'locality': object, 'town_name': str, 
                                  'administrative_area': str, 'post_town': str,  'postcode': str, 
                                  'postcode_locator': str}
"""

address_matching_columns = ['uprn', 'parent_uprn', 'class', 'latitude', 'longitude', 'country',
                    'legal_name','sub_building_name', 'building_name','building_number',
                    'sao_start_number','sao_start_suffix','sao_end_number',
           'sao_end_suffix','sao_text', 'pao_start_number','pao_start_suffix',
           'pao_end_number','pao_end_suffix','pao_text', 'street_description',
                    'dependent_locality','locality', 'town_name', 'administrative_area','post_town','postcode', 'postcode_locator' ]

address_base_file = "/Users/christine/Documents/_UCL_grad school/research/research_pd_24/data/ab_plus_england_202308150944.csv"


In [5]:
#read in the address base data - 32 Million rows
t = time.time()
df_lst = [] 


# Iterate over the file based on the criteria and append to the list
for df in read_csv(address_base_file, address_matching_columns):
       # if df['country'] == 'E':
       # tmp_df = (df.pipe(lambda x:  x[x.country == 'E'] ))
    df_lst +=   [df.copy()] 

# And finally combine filtered df_lst into the final larger output say 'df_final' dataframe 
df_final = pd.concat(df_lst)
print('seconds: ', time.time() - t)

seconds:  404.82963609695435


In [6]:
test = df_final.iloc[:10,:]

In [7]:
test['postcode_sector'] = test['postcode_locator'].apply(lambda x: x[:-2])

/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_75520/939437873.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['postcode_sector'] = test['postcode_locator'].apply(lambda x: x[:-2])


In [9]:
#test.replace([np.nan, -np.inf], '', inplace=True)

/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_60657/860496464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.replace([np.nan, -np.inf], '', inplace=True)


In [13]:
def combine_columns_1(row): 
    columns = ['sao_text','pao_text',  'pao_start_number', 'street_description']
    list_ = [str(row[x]) for x in columns]
   # print(list_)
    return ', '.join(filter(None, list_))

In [14]:
def combine_columns_2(row): 
    columns = ['sub_building_name', 'building_name','building_number', 'street_description']
    list_ = [str(row[x]) for x in columns]
   # print(list_)
    return ', '.join(filter(None, list_))

In [15]:
df_final['building_number'] = df_final['building_number'].astype('Int64').astype('str') 

In [16]:
df_final['building_number'] # print this out to see what's happening... 

0           <NA>
1           <NA>
2           <NA>
3           <NA>
4           <NA>
            ... 
32339409      36
32339410      73
32339411      20
32339412      18
32339413      19
Name: building_number, Length: 32339414, dtype: object

In [17]:
df_final['pao_start_number'] = df_final['pao_start_number'].astype('Int64').astype('str') 

In [18]:
df_final['pao_start_number']

0           <NA>
1           <NA>
2           <NA>
3           <NA>
4             85
            ... 
32339409      36
32339410      73
32339411      20
32339412      18
32339413      19
Name: pao_start_number, Length: 32339414, dtype: object

In [19]:
# here I am replacing the Nans with e
df_final = df_final.replace([np.nan, -np.inf], '')


In [20]:
# add in the replacement for NA (in building number) so that it is just empty string
df_final = df_final.replace('<NA>', '')

KeyboardInterrupt: 

In [28]:
df_final.head()

,uprn,class,parent_uprn,latitude,longitude,country,legal_name,sub_building_name,building_name,building_number,...,pao_end_suffix,pao_text,street_description,dependent_locality,locality,town_name,administrative_area,post_town,postcode,postcode_locator
0,10012778289,RD02,,54.036005,-2.340833,E,,,,,...,,SCOUTBER END FARM,OLD OLIVER LANE TO RAGGED HALL,,RATHMELL,SETTLE,NORTH YORKSHIRE,,,BD24 0LP
1,10013268085,LU01,,51.628915,0.471228,E,,,,,...,,MEEPSWOOD,PARK LANE,,RAMSDEN HEATH,CHELMSFORD,ESSEX,,,CM11 1NN
2,10013819934,RG02,10013819576.0,54.780687,-1.510651,E,,,,,...,,GARAGE SITE 53,DOWSEY ROAD,,,SHERBURN,DURHAM,,,DH6 1JH
3,10014205932,PS,,51.590568,0.600675,E,,,,,...,,STREET RECORD,FOOTPATH FROM GLEBE DRIVE TO THE APPROACH,,,RAYLEIGH,ESSEX,,,SS6 9HJ
4,10014309637,RD04,200003655499.0,51.266510,0.497553,E,,,,,...,,,MILTON STREET,,,MAIDSTONE,KENT,,,ME16 8LD


In [ ]:
df_final 

In [36]:
test = df_final.iloc[:10,:]

In [37]:
#run it to see what it looks like
test['parsed_address1'] = test.apply(combine_columns_1, axis = 1)
test['parsed_address2'] = test.apply(combine_columns_2, axis = 1)

/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_15306/572530206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['parsed_address1'] = test.apply(combine_columns_1, axis = 1)
/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_15306/572530206.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['parsed_address2'] = test.apply(combine_columns_2, axis = 1)


In [38]:
test.head()

,uprn,class,parent_uprn,latitude,longitude,country,legal_name,sub_building_name,building_name,building_number,...,street_description,dependent_locality,locality,town_name,administrative_area,post_town,postcode,postcode_locator,parsed_address1,parsed_address2
0,10012778289,RD02,,54.036005,-2.340833,E,,,,,...,OLD OLIVER LANE TO RAGGED HALL,,RATHMELL,SETTLE,NORTH YORKSHIRE,,,BD24 0LP,"SCOUTBER END FARM, OLD OLIVER LANE TO RAGGED HALL",OLD OLIVER LANE TO RAGGED HALL
1,10013268085,LU01,,51.628915,0.471228,E,,,,,...,PARK LANE,,RAMSDEN HEATH,CHELMSFORD,ESSEX,,,CM11 1NN,"MEEPSWOOD, PARK LANE",PARK LANE
2,10013819934,RG02,10013819576.0,54.780687,-1.510651,E,,,,,...,DOWSEY ROAD,,,SHERBURN,DURHAM,,,DH6 1JH,"GARAGE SITE 53, DOWSEY ROAD",DOWSEY ROAD
3,10014205932,PS,,51.590568,0.600675,E,,,,,...,FOOTPATH FROM GLEBE DRIVE TO THE APPROACH,,,RAYLEIGH,ESSEX,,,SS6 9HJ,"STREET RECORD, FOOTPATH FROM GLEBE DRIVE TO TH...",FOOTPATH FROM GLEBE DRIVE TO THE APPROACH
4,10014309637,RD04,200003655499.0,51.266510,0.497553,E,,,,,...,MILTON STREET,,,MAIDSTONE,KENT,,,ME16 8LD,"ROOM 1, 85, MILTON STREET",MILTON STREET


In [ ]:
# create a function to combine columns 'sub_building', 'building_name', 'building_number', 'street_description'
    # also 'sao_text', 'pao_text', 'pao_start_number', 'street_description'
#def create_address_1

In [39]:
%%time
## Add the column that adds parsed string to the entire dataset...
df_final['parsed_address1'] = df_final.apply(combine_columns_1, axis = 1)
df_final['parsed_address2'] = df_final.apply(combine_columns_2, axis = 1)

CPU times: user 25min 15s, sys: 9min 28s, total: 34min 44s
Wall time: 1h 26min 28s


In [40]:
#data cleaning - #replace nan
#df_final = df_final.replace([np.nan, -np.inf], 0)
resi_AB = df_final[df_final['class'].str.startswith('R')] 
other_AB = df_final[df_final['class'].str.startswith('R') == False] 

In [42]:
resi_AB.head()

,uprn,class,parent_uprn,latitude,longitude,country,legal_name,sub_building_name,building_name,building_number,...,street_description,dependent_locality,locality,town_name,administrative_area,post_town,postcode,postcode_locator,parsed_address1,parsed_address2
0,10012778289,RD02,,54.036005,-2.340833,E,,,,,...,OLD OLIVER LANE TO RAGGED HALL,,RATHMELL,SETTLE,NORTH YORKSHIRE,,,BD24 0LP,"SCOUTBER END FARM, OLD OLIVER LANE TO RAGGED HALL",OLD OLIVER LANE TO RAGGED HALL
2,10013819934,RG02,10013819576.0,54.780687,-1.510651,E,,,,,...,DOWSEY ROAD,,,SHERBURN,DURHAM,,,DH6 1JH,"GARAGE SITE 53, DOWSEY ROAD",DOWSEY ROAD
4,10014309637,RD04,200003655499.0,51.266510,0.497553,E,,,,,...,MILTON STREET,,,MAIDSTONE,KENT,,,ME16 8LD,"ROOM 1, 85, MILTON STREET",MILTON STREET
21,10033213520,RD01,10002821654.0,53.903252,-0.158754,E,,,,,...,HORNSEA BURTON ROAD,,,HORNSEA,EAST RIDING OF YORKSHIRE,,,HU18 1TL,"33 SOUTHFIELD, LONGBEACH LEISURE PARK, HORNSEA...",HORNSEA BURTON ROAD
22,10033545757,RD06,100023479131.0,51.514983,-0.179260,E,,,,,...,WESTBOURNE TERRACE,,,LONDON,CITY OF WESTMINSTER,,,W2 3UJ,"SECOND FLOOR, 58, WESTBOURNE TERRACE",WESTBOURNE TERRACE


In [15]:
#resi_AB.iloc[0].loc['building_number']

'<NA>'

In [43]:
# DATA EXPLORATION - EXPORT CSV IF NEEDED 
resi_AB.to_csv('resi_AB_cleaned_parsed_030524.csv', index = False)

In [44]:
# DATA EXPLORATION - EXPORT CSV IF NEEDED 
other_AB.to_csv('other_AB_cleaned_parsed_030524.csv', index = False)

In [ ]:
#basically, we want to create an extra column with the address... 
"""
legal_name                                        0.0
sub_building_name                                   0
building_name                                       0
building_number                                   0.0
sao_start_number                                  0.0
sao_start_suffix                                    0
sao_end_number                                    0.0
sao_end_suffix                                      0
sao_text                                            0
pao_start_number                                  0.0
pao_start_suffix                                    0
pao_end_number                                    0.0
pao_end_suffix                                      0
pao_text                            SCOUTBER END FARM
street_description     OLD OLIVER LANE TO RAGGED HALL
"""